In [21]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GenerationConfig
import torch
import torch.nn as nn
import os

from transformers.onnx.features import FeaturesManager

In [2]:
import numpy as np

def save_to(base, x, y):
    np_input = x.detach().numpy()
    with open(os.path.join(base, 'dummy_in.dat'), 'wb') as f:
        np_input.tofile(f)

    np_output = y.detach().numpy()
    with open(os.path.join(base, 'dummy_out.dat'), 'wb') as f:
        np_output.tofile(f)

In [3]:
torch.manual_seed(2333)
torch.set_num_threads(1)

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.return_dict = False
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

In [13]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [19]:
gen_cfg = GenerationConfig(
    pad_token_id = tokenizer.eos_token_id, 
    use_cache = True,
    max_new_tokens = 128)

In [20]:
%%time
output = model.generate(**encoded_input, generation_config=gen_cfg)
print(tokenizer.decode(output[0]))

Replace me by any text you'd like.

I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you
CPU times: user 5.14 s, sys: 264 ms, total: 5.41 s
Wall time: 5.46 s


In [ ]:
gpt2_features = list(FeaturesManager.get_supported_features_for_model_type("gpt2").keys())

In [5]:
input_names = ["text"]
output_names = ["prediction"]
dummy_input = torch.randint(0, 30522, (64, 20))

torch.onnx.export(bert,
                  dummy_input,
                  "./bert.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [10]:
save_to('.', x, y.last_hidden_state)